In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:

model_name = 'huggyllama/llama-7b'
tokenize = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
true_summary = "According to the Commission's complaint, the defendants raised at least $23.4 million from thousands of investors in the Haitian-American community nationwide through a network of purported investment clubs Theodule directs investors to form."
true_text = """The Defendants have engaged in a fraudulent Ponzi scheme primarily targeting the US Haitian community since at least November 2007.This includes net transfers of at least $1.7 million to his personal bank accounts, cash withdrawals of more than $1.5 million and more than $600,000 for apparent personal expenses such as two luxury vehicles, credit card bills, a wedding payment, and a house down payment.21. The investment clubs pool investor funds and send them to Creative Capital for a 90-day period, during which Theodule purportedly trades stocks and options on behalf of the investment club members.Page 2 of 10 
$15.2 million collected from new investors in typical Ponzi scheme fashion.Page 4 of 10 
made millionaires out of a significant number of people in the time it had taken her to decide to invest, and pressured her to liquidate the equity in her home to invest with him.14. Theodule ingratiates himself with investors by claiming he recently decided to offer his investment expertise to help build wealth in the Haitian community.He also tells investors he uses part of his trading profits to fund start-up businesses in the Haitian community, as well as business projects in Haiti and Sierra Leone.15. The Defendants primarily attract investors through word-of-mouth, and Theodule makes his representations during face-to-face meetings in which he touts his ability to double investor funds in just 90 days.18. Since the commencement of the investment scheme, the Defendants have raised more than $23.4 million from thousands of investors nationwide.2, From at least November 2007 to the present, Theodule, directly and through the Companies, has raised at least $23.4 million from thousands of investors in an ongoing fraud and Ponzi scheme targeting mostly Haitian and Haitian-American investors nationwide."""

false_summary = " According to the Commission's complaint, the defendants raised at least $789 million from thousands of investors in the Haitian-American community nationwide through a network of purported investment clubs Theodule directs investors to form."
false_text = """The Defendants have engaged in a fraudulent Ponzi scheme primarily targeting the US Haitian community since at least November 2007.This includes net transfers of at least $1.7 million to his personal bank accounts, cash withdrawals of more than $1.5 million and more than $600,000 for apparent personal expenses such as two luxury vehicles, credit card bills, a wedding payment, and a house down payment.21. The investment clubs pool investor funds and send them to Creative Capital for a 90-day period, during which Theodule purportedly trades stocks and options on behalf of the investment club members.Page 2 of 10 
$15.2 million collected from new investors in typical Ponzi scheme fashion.Page 4 of 10 
made millionaires out of a significant number of people in the time it had taken her to decide to invest, and pressured her to liquidate the equity in her home to invest with him.14. Theodule ingratiates himself with investors by claiming he recently decided to offer his investment expertise to help build wealth in the Haitian community.He also tells investors he uses part of his trading profits to fund start-up businesses in the Haitian community, as well as business projects in Haiti and Sierra Leone.15. The Defendants primarily attract investors through word-of-mouth, and Theodule makes his representations during face-to-face meetings in which he touts his ability to double investor funds in just 90 days.18. Since the commencement of the investment scheme, the Defendants have raised more than $23.4 million from thousands of investors nationwide.2, From at least November 2007 to the present, Theodule, directly and through the Companies, has raised at least $23.4 million from thousands of investors in an ongoing fraud and Ponzi scheme targeting mostly Haitian and Haitian-American investors nationwide."""

true_input_text = f"""Evaluate the compliance of a summary sentence derived from a set of sentences in a financial document. Adhere to the following verification standards:
1. Entity consistency: Check that all named entities in the summary are extracted from the source.
2. Relationship verification: Confirm that relationships between entities in the summary are present and correctly depicted in the source.
3. Directionality check: Ensure that the direction of relationships between entities in the summary matches those in the source.
4. Factual integrity: Ascertain that the summary is free from factual errors when compared to the source.
5. Entity authenticity: Confirm that the summary does not create non-existent entities.

Based on these criteria, determine if the summary sentence is a faithful representation of the source sentences. Respond with "True" if the summary complies with all standards, or "False" if it does not.

Summary sentence: {true_summary}

Source sentences: {true_text}"""

false_input_text = f"""Evaluate the compliance of a summary sentence derived from a set of sentences in a financial document. Adhere to the following verification standards:
1. Entity consistency: Check that all named entities in the summary are extracted from the source.
2. Relationship verification: Confirm that relationships between entities in the summary are present and correctly depicted in the source.
3. Directionality check: Ensure that the direction of relationships between entities in the summary matches those in the source.
4. Factual integrity: Ascertain that the summary is free from factual errors when compared to the source.
5. Entity authenticity: Confirm that the summary does not create non-existent entities.

Based on these criteria, determine if the summary sentence is a faithful representation of the source sentences. Respond with "True" if the summary complies with all standards, or "False" if it does not.

Summary sentence: {false_summary}

Source sentences: {false_text}"""


In [4]:
true_input_text = f"""please tell me whether or not the following summary sentence is faithful to the given source sentence. answer "yes" if it is, and "no" if it is not. Here are the summary and source sentences:
summary sentence: ```{true_summary}```
source sentences: ```{true_text}```"""
false_input_text = f"""please tell me whether or not the following summary sentence is faithful to the given source sentence. answer "yes" if it is, and "no" if it is not. Here are the summary and source sentences:
summary sentence: ```{false_summary}```
source sentences: ```{false_text}```"""

In [5]:

true_input = tokenize(true_input_text, return_tensors='pt')
false_input = tokenize(false_input_text, return_tensors='pt')

True_idx = tokenize.encode("yes", add_special_tokens=False)[0]
False_idx = tokenize.encode("no", add_special_tokens=False)[0]

In [6]:
true_output = model(**true_input)
true_logits = true_output.logits

In [7]:
prob = torch.softmax(true_logits[:, -1, :], dim=1)
yes_prob = prob[:, True_idx].item()
no_prob = prob[:, False_idx].item()
print(yes_prob, no_prob)
print(yes_prob > no_prob)

1.2404984772729222e-05 3.2171892598853447e-06
True


In [8]:
false_output = model(**false_input)
false_logits = false_output.logits
false_prob = torch.softmax(false_logits[:, -1, :], dim=1)
yes_false_prob = false_prob[:, True_idx].item()
no_false_prob = false_prob[:, False_idx].item()
print(yes_false_prob, no_false_prob)
print(yes_false_prob > no_false_prob)


1.79980543180136e-05 5.803504791401792e-06
True
